<a href="https://colab.research.google.com/github/Soumadeep555/RAG/blob/main/Multimodal_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install the required libraries:

In [ ]:
! pip install "unstructured[all-docs]" pillow pydantic lxml matplotlib

In [ ]:
!sudo apt-get update

In [ ]:
!sudo apt-get install poppler-utils

In [ ]:
!sudo apt-get install tesseract-ocr

In [ ]:
!sudo apt-get install libtesseract-dev

In [ ]:
!pip install unstructured-pytesseract

## Data Parsing / Data Extraction:

In [ ]:
from unstructured.partition.pdf import partition_pdf

In [ ]:
raw_pdf_elements=partition_pdf(
    filename="/content/drive/MyDrive/Generative_AI_Advanced_NLP/Advance_NLP_Generative_AI/CODE/data/openai.pdf",
    strategy="hi_res",
    extract_images_in_pdf=True,
    extract_image_block_types=["Image","Table"],
    extract_image_block_to_payload=False,
    extract_image_block_output_dir="/content/drive/MyDrive/Generative_AI_Advanced_NLP/Advance_NLP_Generative_AI/CODE/data/extracted_data"
    )

In [ ]:
raw_pdf_elements

In [ ]:
Header=[]
Footer=[]
Title=[]
NarrativeText=[]
Text=[]
ListItem=[]
Image=[]
Table=[]

for element in raw_pdf_elements:
  if "unstructured.documents.elements.Header" in str(type(element)):
            Header.append(str(element))
  elif "unstructured.documents.elements.Footer" in str(type(element)):
            Footer.append(str(element))
  elif "unstructured.documents.elements.Title" in str(type(element)):
            Title.append(str(element))
  elif "unstructured.documents.elements.NarrativeText" in str(type(element)):
            NarrativeText.append(str(element))
  elif "unstructured.documents.elements.Text" in str(type(element)):
            Text.append(str(element))
  elif "unstructured.documents.elements.ListItem" in str(type(element)):
            ListItem.append(str(element))
  elif "unstructured.documents.elements.Image" in str(type(element)):
            Image.append(str(element))
  elif "unstructured.documents.elements.Table" in str(type(element)):
            Table.append(str(element))

In [ ]:
Header

In [ ]:
Footer

In [ ]:
ListItem

In [ ]:
Title

In [ ]:
NarrativeText

In [ ]:
NarrativeText[0]

In [ ]:
NarrativeText[1]

In [ ]:
Text

In [ ]:
Table

In [ ]:
Image

## Load the Model and Embedding Model:

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')